In [ ]:
'''
Transfer learning is a machine learning technique where a pre-trained model (trained on a large dataset and task) is reused as the starting point for a new but related task.
'''

In [1]:
!pip install kagglehub


   ---------------------------------------- 0/8 [urllib3]
   ----- ---------------------------------- 1/8 [tqdm]
   ---------- ----------------------------- 2/8 [pyyaml]
   --------------- ------------------------ 3/8 [idna]
   ------------------------------ --------- 6/8 [requests]
   ----------------------------------- ---- 7/8 [kagglehub]
   ---------------------------------------- 8/8 [kagglehub]



In [4]:
import os
import torch.nn as nn
import torch.nn.functional as F
import torch
import shutil
import kagglehub
from torchvision import models
import torchvision.models as models
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split,DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
# Download the dataset
path = kagglehub.dataset_download("tawsifurrahman/covid19-radiography-database")
path

100%|██████████| 778M/778M [03:04<00:00, 4.42MB/s] 

Extracting files...


'C:\\Users\\tahsi\\.cache\\kagglehub\\datasets\\tawsifurrahman\\covid19-radiography-database\\versions\\5'

In [ ]:
# Define source and destination folder
source_root = os.path.join(path,'COVID-19_Radiography_Dataset')
target_root = 'C:/Pytorch Practice/Data/COVID19_CLASSIFICATION'
classes = ['COVID', 'Normal', 'Lung_Opacity', 'Viral Pneumonia']
for cls in classes:
  src_folder = os.path.join(source_root,cls,'images')
  dst_folder = os.path.join(target_root,cls)
  os.makedirs(dst_folder,exist_ok=True)

  for file in os.listdir(src_folder):
    src_file = os.path.join(src_folder,file)
    dst_file = os.path.join(dst_folder,file)
    if os.path.isfile(src_file):
      shutil.copy2(src_file,dst_file)


In [ ]:
transform = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor(),
])
dataset = datasets.ImageFolder(root='C:/Pytorch Practice/Data/COVID19_CLASSIFICATION',transform=transform)

In [ ]:
train_size = int(0.8*len(dataset))
val_size = len(dataset)- train_size

# Split train and validation dataset
train_dataset, val_dataset = random_split(dataset,[train_size,val_size])

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=False)
# define device and model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:

# Define model, loss function and optimizer
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features,4)  # Just modify last fully connected layer 
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.0001)
num_epochs = 10


print('Pretrained Resnet model\n')
# Training and validation 
for epoch in range(num_epochs):
  model.train()
  running_loss = 0.0
  correct,total = 0,0

  for inputs,labels in train_loader:
    inputs, labels = inputs.to(device),labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    running_loss+= loss.item()
    _,pred = outputs.max(1)
    correct +=(pred==labels).sum().item()
    total+= labels.size(0)

  train_acc = 100* correct/total

  # validation
  model.eval()
  val_correct = 0
  val_total = 0
  with torch.no_grad():
    for inputs, labels in val_loader:
      inputs, labels = inputs.to(device),labels.to(device)
      outputs = model(inputs)
      _,pred = outputs.max(1)
      val_correct +=(pred==labels).sum().item()
      val_total+= labels.size(0)

  val_acc = 100* val_correct/val_total

  print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {running_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

